In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.metrics import confusion_matrix
import time

class ModelComparisonVisualizer:
    def __init__(self):
        self.models = {
            'B3': {'color': 'blue', 'marker': 'o', 'offset': -0.25},
            'B4': {'color': 'green', 'marker': 's', 'offset': 0},
            'B5': {'color': 'red', 'marker': '^', 'offset': 0.25}
        }
        self.class_names = ['Sky', 'Building', 'Pole', 'Road', 'Sidewalk', 
                           'Tree', 'Sign', 'Fence', 'Car', 'Pedestrian', 'Background']
        
        # Create results directory
        os.makedirs('comparison_results', exist_ok=True)

    def parse_training_log(self, log_content):
        """Parse training log content to extract train and validation losses"""
        train_losses = []
        val_losses = []
        
        try:
            for line in log_content.split('\n'):
                if 'Train Loss:' in line:
                    try:
                        train_loss = float(line.split('Train Loss:')[1].strip())
                        train_losses.append(train_loss)
                    except ValueError:
                        continue
                elif 'Val Loss:' in line and 'New best model' not in line:
                    try:
                        val_loss = float(line.split('Val Loss:')[1].strip())
                        val_losses.append(val_loss)
                    except ValueError:
                        continue
        except Exception as e:
            print(f"Error parsing log: {str(e)}")
            
        return {'train_loss': train_losses, 'val_loss': val_losses}

    def load_training_data(self, b3_log, b4_log, b5_log):
            """Load and parse training data for all models"""
            self.training_data = {
                'B3': self.parse_training_log(b3_log),
                'B4': self.parse_training_log(b4_log),
                'B5': self.parse_training_log(b5_log)
            }

    def plot_training_curves(self, save_path='comparison_results/training_curves.png'):
        """Plot training and validation loss curves for all models"""
        plt.figure(figsize=(12, 6))
        
        for model_name, data in self.training_data.items():
            # Plot training loss
            plt.plot(data['train_loss'], 
                    label=f'{model_name} Train',
                    color=self.models[model_name]['color'],
                    linestyle='-', 
                    marker=self.models[model_name]['marker'],
                    markevery=5)
            
            # Plot validation loss
            plt.plot(data['val_loss'], 
                    label=f'{model_name} Val',
                    color=self.models[model_name]['color'],
                    linestyle='--', 
                    marker=self.models[model_name]['marker'],
                    markevery=5)

        plt.title('Training and Validation Loss Comparison')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    def plot_model_statistics(self, save_path='comparison_results/model_statistics.png'):
        """Plot model statistics comparison"""
        # Define static statistics for B3, B4, B5
        model_stats = {
            'B3': {'parameters': 47.1, 'model_size': 180.2, 'inference_time': 25.3, 'gpu_memory': 4.2},
            'B4': {'parameters': 64.1, 'model_size': 244.7, 'inference_time': 28.5, 'gpu_memory': 5.1},
            'B5': {'parameters': 84.7, 'model_size': 322.5, 'inference_time': 32.8, 'gpu_memory': 6.3}
        }

        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # Parameters plot
        params = [stats['parameters'] for stats in model_stats.values()]
        ax1.bar(model_stats.keys(), params)
        ax1.set_title('Number of Parameters (M)')
        ax1.grid(True, alpha=0.3)

        # Model size plot
        sizes = [stats['model_size'] for stats in model_stats.values()]
        ax2.bar(model_stats.keys(), sizes)
        ax2.set_title('Model Size (MB)')
        ax2.grid(True, alpha=0.3)

        # Inference time plot
        times = [stats['inference_time'] for stats in model_stats.values()]
        ax3.bar(model_stats.keys(), times)
        ax3.set_title('Inference Time (ms)')
        ax3.grid(True, alpha=0.3)

        # Memory usage plot
        memory = [stats['gpu_memory'] for stats in model_stats.values()]
        ax4.bar(model_stats.keys(), memory)
        ax4.set_title('GPU Memory Usage (GB)')
        ax4.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    def plot_best_performance(self, save_path='comparison_results/best_performance.png'):
        # """Plot best performance metrics for each model"""
        best_metrics = {
            'B3': {'val_loss': 0.6197, 'mIoU': 77.9, 'epoch': 25},
            'B4': {'val_loss': 0.6576, 'mIoU': 78.5, 'epoch': 12},
            'B5': {'val_loss': 0.6291, 'mIoU': 82.4, 'epoch': 19}
        }

        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
        
        # Best validation loss comparison
        val_losses = [metrics['val_loss'] for metrics in best_metrics.values()]
        ax1.bar(best_metrics.keys(), val_losses)
        ax1.set_title('Best Validation Loss')
        ax1.grid(True, alpha=0.3)

        # mIoU comparison
        mious = [metrics['mIoU'] for metrics in best_metrics.values()]
        ax2.bar(best_metrics.keys(), mious)
        ax2.set_title('Mean IoU (%)')
        ax2.grid(True, alpha=0.3)

        # Epochs to convergence
        epochs = [metrics['epoch'] for metrics in best_metrics.values()]
        ax3.bar(best_metrics.keys(), epochs)
        ax3.set_title('Epochs to Best Performance')
        ax3.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    def plot_advanced_comparisons(self, save_path_prefix='comparison_results'):
        """Comprehensive advanced comparisons for B3, B4, and B5"""
        
        # 1. Efficiency Metrics
        efficiency_metrics = {
            'B3': {
                'miou_per_param': 77.9/47.1,
                'miou_per_time': 77.9/25.3,
                'miou_per_memory': 77.9/4.2
            },
            'B4': {
                'miou_per_param': 78.5/64.1,
                'miou_per_time': 78.5/28.5,
                'miou_per_memory': 78.5/5.1
            },
            'B5': {
                'miou_per_param': 82.4/84.7,
                'miou_per_time': 82.4/32.8,
                'miou_per_memory': 82.4/6.3
            }
        }
    
        # Plot efficiency comparison
        plt.figure(figsize=(15, 6))
        metrics = ['miou_per_param', 'miou_per_time', 'miou_per_memory']
        x = np.arange(len(metrics))
        width = 0.25
    
        for i, (model, data) in enumerate(efficiency_metrics.items()):
            values = [data[m] for m in metrics]
            plt.bar(x + i*width, values, width, label=model)
    
        plt.xlabel('Efficiency Metrics')
        plt.ylabel('Score')
        plt.title('Efficiency Comparison')
        plt.xticks(x + width, ['mIoU/Parameters', 'mIoU/InferenceTime', 'mIoU/Memory'])
        plt.legend()
        plt.savefig(f'{save_path_prefix}/efficiency_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
    
        # 2. Performance vs Resource Usage
        plt.figure(figsize=(12, 8))
        models = ['B3', 'B4', 'B5']
        params = [47.1, 64.1, 84.7]
        mious = [77.9, 78.5, 82.4]
        sizes = [100, 150, 200]  # Size of scatter points proportional to GPU memory
    
        plt.scatter(params, mious, s=sizes, alpha=0.6)
        for i, model in enumerate(models):
            plt.annotate(model, (params[i], mious[i]))
    
        plt.xlabel('Parameters (M)')
        plt.ylabel('Mean IoU (%)')
        plt.title('Performance vs Model Size')
        plt.grid(True, alpha=0.3)
        plt.savefig(f'{save_path_prefix}/performance_vs_size.png', dpi=300, bbox_inches='tight')
        plt.close()
    
        # 3. Spider/Radar Plot for Multiple Metrics
        metrics = ['Parameter Efficiency', 'Speed', 'Memory Efficiency', 
                  'mIoU', 'Convergence Speed']
        
        # Normalized scores for each model (scale of 0-1)
        model_scores = {
            'B3': [0.9, 0.95, 0.95, 0.85, 0.8],
            'B4': [0.85, 0.9, 0.9, 0.88, 0.85],
            'B5': [0.8, 0.85, 0.85, 1.0, 0.9]
        }
    
        angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False)
        
        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
        
        for model, scores in model_scores.items():
            values = np.concatenate((scores, [scores[0]]))  # complete the circle
            angles_plot = np.concatenate((angles, [angles[0]]))
            ax.plot(angles_plot, values, 'o-', label=model)
            ax.fill(angles_plot, values, alpha=0.25)
        
        ax.set_xticks(angles)
        ax.set_xticklabels(metrics)
        ax.set_title('Multi-dimensional Performance Analysis')
        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        plt.savefig(f'{save_path_prefix}/radar_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()
    
        # 4. Training Stability Analysis
        plt.figure(figsize=(12, 6))
        window_size = 5
        
        for model_name, data in self.training_data.items():
            val_losses = np.array(data['val_loss'])
            stability = np.array([np.std(val_losses[max(0, i-window_size):i+1]) 
                                for i in range(len(val_losses))])
            
            plt.plot(stability, label=f'{model_name} Stability',
                    color=self.models[model_name]['color'])
    
        plt.xlabel('Epochs')
        plt.ylabel('Loss Stability (Rolling StdDev)')
        plt.title('Training Stability Analysis')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(f'{save_path_prefix}/training_stability.png', dpi=300, bbox_inches='tight')
        plt.close()

    def create_summary_table(self, save_path='comparison_results/summary.csv'):
        # """Create comprehensive summary table"""
        summary = {
            'Model': ['B3', 'B4', 'B5'],
            'Parameters (M)': [47.1, 64.1, 84.7],
            'Model Size (MB)': [180.2, 244.7, 322.5],
            'Best Val Loss': [0.6197, 0.6576, 0.6291],
            'Mean IoU (%)': [77.9, 78.5, 82.4],
            'Inference Time (ms)': [25.3, 28.5, 32.8],
            'GPU Memory (GB)': [4.2, 5.1, 6.3]
        }
        
        df = pd.DataFrame(summary)
        df.to_csv(save_path, index=False)
        return df

    def run_complete_analysis(self, b3_log, b4_log, b5_log):
        # """Run complete analysis and generate all visualizations"""
        try:
            # Load and process training data
            self.load_training_data(b3_log, b4_log, b5_log)
            
            # Generate all visualizations
            self.plot_training_curves()
            self.plot_model_statistics()
            self.plot_best_performance()
            self.plot_advanced_comparisons()
            
            # Create summary table
            summary_df = self.create_summary_table()
            
            print("Analysis completed successfully!")
            return summary_df
            
        except Exception as e:
            print(f"Error during analysis: {str(e)}")
            return None

# Example usage
if __name__ == "__main__":
    visualizer = ModelComparisonVisualizer()
    
    try:
        # Read log files
        with open('SegFormerB3_Output_Script.txt', 'r') as f:
            b3_log = f.read()
        with open('SegFormerB4_Output_Script.txt', 'r') as f:
            b4_log = f.read()
        with open('SegFormerB5_Output_Script.txt', 'r') as f:
            b5_log = f.read()
        
        # Run complete analysis
        summary = visualizer.run_complete_analysis(b3_log, b4_log, b5_log)
        
        if summary is not None:
            print("\nSummary of Results:")
            print(summary)
            
    except FileNotFoundError as e:
        print(f"Error reading log files: {str(e)}")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")

Analysis completed successfully!

Summary of Results:
  Model  Parameters (M)  Model Size (MB)  Best Val Loss  Mean IoU (%)  \
0    B3            47.1            180.2         0.6197          77.9   
1    B4            64.1            244.7         0.6576          78.5   
2    B5            84.7            322.5         0.6291          82.4   

   Inference Time (ms)  GPU Memory (GB)  
0                 25.3              4.2  
1                 28.5              5.1  
2                 32.8              6.3  


In [ ]:
import glob
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from matplotlib.gridspec import GridSpec

# Test image and class dictionary paths
TEST_IMAGE_PATH = "/home/thatkar/projects/def-saadi/thatkar/CamVid/test/0001TP_006690.png"
CLASS_DICT_PATH = "/home/thatkar/projects/def-saadi/thatkar/CamVid/class_dict.csv"

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load class definitions
class_df = pd.read_csv(CLASS_DICT_PATH)

# Get label path
LABEL_PATH = TEST_IMAGE_PATH.replace('/test/', '/test_labels/').replace('.png', '_L.png')

# Define model paths for each variant
model_paths = {
    "B3": "/home/thatkar/projects/def-saadi/thatkar/CamVid/checkpoints/best_model_loss_b3_0.6197.pth",
    "B4": "/home/thatkar/projects/def-saadi/thatkar/CamVid/checkpoints/best_model_loss_b4_0.6576.pth",
    "B5": "/home/thatkar/projects/def-saadi/thatkar/CamVid/checkpoints/best_model_loss_0.6291.pth"
}

def load_model_and_predict(variant, model_path, image, orig_h, orig_w, device):
    # Initialize feature extractor for the specific variant
    feature_extractor = SegformerImageProcessor.from_pretrained(
        f"nvidia/mit-{variant.lower()}",
        do_reduce_labels=True,
        do_rescale=False,
        size={"height": 640, "width": 640}
    )
    
    # Initialize model for the specific variant
    model = SegformerForSemanticSegmentation.from_pretrained(
        f"nvidia/mit-{variant.lower()}",
        num_labels=len(class_df),
        ignore_mismatched_sizes=True
    ).to(device)
    
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Prepare image tensor
    test_transform = A.Compose([
        A.Resize(height=640, width=640),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    transformed = test_transform(image=image)
    image_tensor = transformed['image'].unsqueeze(0)
    
    # Get prediction
    model.eval()
    with torch.no_grad():
        outputs = model(pixel_values=image_tensor.to(device))
        logits = outputs.logits
        upsampled_logits = nn.functional.interpolate(
            logits,
            size=(orig_h, orig_w),
            mode="bilinear",
            align_corners=False
        )
        predicted = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return predicted

def create_precisely_aligned_visualization():
    # Read images
    image = cv2.imread(TEST_IMAGE_PATH)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    label = cv2.imread(LABEL_PATH)
    orig_h, orig_w = image.shape[:2]
    
    # Get predictions for each model variant
    predictions = {}
    for variant, model_path in model_paths.items():
        print(f"Processing {variant} model...")
        predictions[variant] = load_model_and_predict(variant, model_path, image, orig_h, orig_w, device)
    
    # Create truth mask once
    truth_mask = np.zeros_like(image)
    for idx, row in class_df.iterrows():
        class_color = np.array([row['r'], row['g'], row['b']])
        bgr_color = (int(row['b']), int(row['g']), int(row['r']))
        truth_pixels = np.all(label == bgr_color, axis=2)
        truth_mask[truth_pixels] = class_color
    
    # Create figure with no padding between subplots
    fig = plt.figure(figsize=(12, 9))
    gs = GridSpec(3, 3, figure=fig, wspace=0.01, hspace=0.01)
    
    # Column headers - perfectly aligned with each column
    col_positions = [0.17, 0.5, 0.83]  # Horizontal centers of the three columns
    headers = ['Original Image', 'Ground Truth', 'Model Prediction']
    
    for i, (pos, header) in enumerate(zip(col_positions, headers)):
        plt.figtext(pos, 0.99, header, ha='center', va='top', fontsize=12)
    
    # Position for row labels (centered vertically for each row)
    row_positions = [0.83, 0.5, 0.17]  # Vertical centers of the three rows
    
    for row_idx, variant in enumerate(["B3", "B4", "B5"]):
        # Create prediction mask
        pred_mask = np.zeros_like(image)
        for idx, row in class_df.iterrows():
            class_color = np.array([row['r'], row['g'], row['b']])
            pred_mask[predictions[variant] == idx] = class_color
        
        # Row label - precisely centered vertically for each row
        plt.figtext(0.025, row_positions[row_idx], f'SegFormer-{variant}', 
                  rotation=90, fontsize=12, fontweight='bold',
                  ha='center', va='center')
        
        # Original image
        ax_left = fig.add_subplot(gs[row_idx, 0])
        ax_left.imshow(image)
        ax_left.set_xticks([])
        ax_left.set_yticks([])
        
        # Ground truth
        ax_middle = fig.add_subplot(gs[row_idx, 1])
        ax_middle.imshow(truth_mask)
        ax_middle.set_xticks([])
        ax_middle.set_yticks([])
        
        # Model prediction with variant indicator
        ax_right = fig.add_subplot(gs[row_idx, 2])
        ax_right.imshow(pred_mask)
        ax_right.set_xticks([])
        ax_right.set_yticks([])
        
        # Add (B3), (B4), (B5) labels directly on the prediction images
        # Position at top-right corner of each prediction image
        ax_right.text(0.9, 0.1, f'({variant})', transform=ax_right.transAxes,
                     ha='right', va='top', fontsize=10, color='black',
                     bbox=dict(facecolor='lightgray', alpha=0.7, edgecolor='none', pad=2))
    
    # Adjust layout to be tight with minimal margins
    plt.subplots_adjust(left=0.045, right=0.99, top=0.96, bottom=0.01)
    
    plt.savefig('segformer_perfect_alignment.png', dpi=300, bbox_inches='tight')
    plt.show()

# Run visualization
create_precisely_aligned_visualization()

Processing B3 model...


/home/thatkar/.local/lib/python3.11/site-packages/transformers/utils/deprecation.py:172: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably

Processing B4 model...


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b4 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing B5 model...


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
